# IIoT Minimal Simulation
This notebook consolidates the sensor, monitor, and visualization logic so it can be run directly in Google Colab.

In [ ]:
!pip install matplotlib

## Sensor Definition

In [ ]:
import random
import time

class Sensor:
    """Simulates a basic sensor producing measurements."""
    def __init__(self, name: str, unit: str, min_value: float, max_value: float, interval: float = 1.0):
        self.name = name
        self.unit = unit
        self.min_value = min_value
        self.max_value = max_value
        self.interval = interval
        self.value = None
        self.last_time = None

    def read(self):
        """Generate a new measurement."""
        self.value = random.uniform(self.min_value, self.max_value)
        self.last_time = time.time()
        return self.value


## Monitor Definition

In [ ]:
import time
from collections import defaultdict
from typing import List, Tuple

from .sensor import Sensor

class Monitor:
    """Collects data from sensors at fixed intervals."""
    def __init__(self, sensors: List[Sensor]):
        self.sensors = sensors
        self.data = defaultdict(list)  # sensor name -> list of (timestamp, value)

    def sample(self):
        """Collect a sample from each sensor."""
        for sensor in self.sensors:
            value = sensor.read()
            self.data[sensor.name].append((sensor.last_time, value))

    def run(self, duration: float, interval: float = 1.0):
        """Run sampling for a duration in seconds."""
        end_time = time.time() + duration
        while time.time() < end_time:
            self.sample()
            time.sleep(interval)
        return self.data


## Visualization and Simulation

In [ ]:

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Sensor and Monitor classes are expected to be defined above

def main():
    sensors = [
        Sensor("Water Level", "m", 0.0, 10.0),
        Sensor("Flow Rate", "m3/s", 0.0, 5.0),
        Sensor("pH", "", 6.0, 9.0),
    ]
    monitor = Monitor(sensors)

    fig, ax = plt.subplots()
    lines = {sensor.name: ax.plot([], [], label=sensor.name)[0] for sensor in sensors}
    ax.set_xlim(0, 60)
    ax.set_ylim(0, 10)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Value")
    ax.legend()

    start_time = None

    def update(frame):
        nonlocal start_time
        if start_time is None:
            start_time = frame
        monitor.sample()
        elapsed = frame - start_time
        for name, line in lines.items():
            data = monitor.data[name]
            times = [t - start_time for t, _ in data]
            values = [v for _, v in data]
            line.set_data(times, values)
        ax.set_xlim(max(0, elapsed - 60), elapsed + 5)
        return list(lines.values())

    ani = FuncAnimation(fig, update, interval=1000)
    plt.show()


Execute `main()` below to launch the real-time visualization.